In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Load the FER2013 dataset from CSV
data = pd.read_csv('fer2013.csv')

# Initialize lists to store valid data
valid_pixels = []
valid_emotions = []

# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    # Try to convert the pixel string to an array
    try:
        pixels = np.fromstring(row['pixels'], dtype=int, sep=' ')
        # Check if the length matches the expected size (48*48 = 2304)
        if len(pixels) == 2304:
            valid_pixels.append(pixels)
            valid_emotions.append(row['emotion'])
    except Exception as e:
        # Skip the row if there's an error
        print(f"Error processing row {index}: {e}")

# Convert lists to numpy arrays
pixels = np.array(valid_pixels)
emotions = pd.get_dummies(valid_emotions).values

# Normalize pixel values
pixels = pixels / 255.0

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pixels, emotions, test_size=0.1, random_state=42)

# Reshape the flattened pixel arrays to images
X_train_images = X_train.reshape(-1, 48, 48, 1)
X_test_images = X_test.reshape(-1, 48, 48, 1)

# Expand the dimensions to match the input shape of VGG16
X_train_images = np.repeat(X_train_images, 3, axis=-1)
X_test_images = np.repeat(X_test_images, 3, axis=-1)


# Load pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(7, activation='softmax')(x)

# Create the model
model = Model(base_model.input, x)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define a checkpoint callback to save the best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
# Train the model with the reshaped input data
model.fit(X_train_images, y_train, batch_size=32, epochs=20, validation_data=(X_test_images, y_test), callbacks=[checkpoint])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_images, y_test)
print('Test Accuracy:', accuracy)



Epoch 1/20
187/187 [==============================] - ETA: 0s - loss: 1.7020 - accuracy: 0.3214
Epoch 1: val_accuracy improved from -inf to 0.35445, saving model to best_model.h5
187/187 [==============================] - 128s 681ms/step - loss: 1.7020 - accuracy: 0.3214 - val_loss: 1.6430 - val_accuracy: 0.3544
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


187/187 [==============================] - ETA: 0s - loss: 1.5738 - accuracy: 0.3819
Epoch 2: val_accuracy improved from 0.35445 to 0.37104, saving model to best_model.h5
187/187 [==============================] - 127s 679ms/step - loss: 1.5738 - accuracy: 0.3819 - val_loss: 1.6061 - val_accuracy: 0.3710
Epoch 3/20
187/187 [==============================] - ETA: 0s - loss: 1.5267 - accuracy: 0.4031
Epoch 3: val_accuracy did not improve from 0.37104
187/187 [==============================] - 125s 671ms/step - loss: 1.5267 - accuracy: 0.4031 - val_loss: 1.5980 - val_accuracy: 0.3695
Epoch 4/20
187/187 [==============================] - ETA: 0s - loss: 1.4705 - accuracy: 0.4346
Epoch 4: val_accuracy improved from 0.37104 to 0.37406, saving model to best_model.h5
187/187 [==============================] - 126s 675ms/step - loss: 1.4705 - accuracy: 0.4346 - val_loss: 1.5937 - val_accuracy: 0.3741
Epoch 5/20
187/187 [==============================] - ETA: 0s - loss: 1.4189 - accuracy: 0.4609